# 15.1 关于 else：if 之外的 else

* for 仅当 for 循环运行完毕时(即 for 循环没有被 break 语句中止)才运行 else 模块
* while 仅当 while 循环因为条件为假值而退出时(即 while 循环没有被 break 语句中止) 才运行 else 块
* try 仅当 try 块中没有异常抛出时才运行 else 块。

在所有情况下，如果异常或者 return、break 或 continue 导致控制器跳到了复合语句的主块之外，else 子句也会被跳过

# 15.2 上下文管理和with块

上下文管理器协议包含 \_\_enter__ 和 \_\_exit__ 两个方法，with 语句开始运行时，会在上下文管理器对象上调用 \_\_enter__ 方法。with 语句运行结束后，会在上下文管理器对象上调用 \_\_exit__ 方法，以此扮演 finally 子句的角色.

执行 with 后面的表达式得到的结果是上下文管理器对象，不过，把值绑定到目标变量上 (as 语句) 上在上下文管理器对象上调用 \_\_enter__ 方法的结果。

不管控制流程以哪种方式退出 with 块，都会在上下文管理器对象上调用 \_\_exit__ 方法，而不是在 \_\_enter__ 方法返回的对象上 调用

with 语句的 as 语句是可选的。对 open 函数来说必须加上 as 子句，以便获取文件的引用。不过有些上下文管理器会返回None，因为没有什么有用的对象提供给用户

In [12]:
# LookingGlass 上下文管理器类
class LookingGlass:

    def __enter__(self):    # 除 self 之外，Python 调用 enter 方法不传入任何参数
        import sys
        self.original_write = sys.stdout.write  # 把原来的 sys.stdout.write 方法存在一个实例属性中，供后面使用
        sys.stdout.write = self.reverse_write   # 为 sys.stdout.write 打猴子补丁，替换成自己编写的方法
        return 'JABBERWOCKY'
    
    def reverse_write(self, text):  # 反转text内容，然后调用原来的实现
        self.original_write(text[::-1])
    
    def __exit__(self, exc_type, exc_value, traceback): 
        # 如果一切正常，Python 调用 exit 方法时传入的参数是 None，None，None；
        # 如果抛出异常，那么它们是异常参数
        import sys  # 重复导入模块不会消耗很多资源，因为 Python 会缓存导入的模块
        sys.stdout.write = self.original_write  # 还原方法
        if exc_type == ZeroDivisionError:   # 如果有异常，而且是指定的异常则打印消息
            print("Plezse DO NOT divide by zero")
            return True # 返回 True 告诉解释器异常已经处理了
        # 如果 exit 方法返回 None，或 True 之外的任何值，with 块中的异常都会向上冒泡

解释器传给  \_\_exit__ 方法 如下：
* exc_type : 异常类
* exc_value : 异常实例。有时会有参数传给异常构造方法，例如错误消息，这些参数可以使用 exc_value.args 获取
* traceback : traceback 对象

In [2]:
# 测试 LookingGlass 上下文管理器类
with LookingGlass() as what:    # enter 方法返回的结果绑定在 what 上
    print('Alice')
    print(what)


ecilA
YKCOWREBBAJ


In [13]:
# 在 with 块之外使用 LookingGlass 类
manager = LookingGlass()
manager

In [14]:
monster = manager.__enter__()
monster == 'JABBERWOCKY'

True

In [9]:
monster

'JABBERWOCKY'

In [11]:
manager.__exit__(None, None, None)
monster

'JABBERWOCKY'

# 15.3 contextlib 模块中的实用工具

* closing : 如果对象提供了 close() 方法，但没有实现 \_\_enter__/\_\_exit__ 协议，那么可以使用这个函数构建上下文管理器
* suppress : 构建临时忽略指定异常的上下文管理器
* @contextmanager : 这个装饰器把简单的生成器函数变成上下文管理器，这样就不用创建类去实现管理器协议了
* ContextDecorator : 这是个基类，用于定义基于类的上下文管理器。这种上下文管理器也能用于装饰函数，在受管理的上下文中运行整个函数
* ExitStack : 这个上下文管理器能进入多个 上下文管理器。with 块结束时，ExitStack 按照LIFO顺序调用栈中的各个上下文管理器 \_\_exit__ 方法。
如果事先不知道 with 块要进入多少个上下文管理器，可以使用这个类。例如同时打开任意一个文件列表中的所有文件

# 15.4 使用 @contextmanager

@contextmanager 装饰器能减少创建上下文管理器的样板代码量，因为不用编写一个完整的类，只需要实现一个 yield 语句，生成想让 \_\_enter__ 方法返回的值。

在使用 @contextmanager 装饰的生成器中，yield 语句的作用是把函数的定义体分成两部分 : yield 语句前面的所有代码在 with 块开始执行，yield 语句后面的代码在 with 块结束时执行。

In [ ]:
# 使用 @contextmanager 实现上下文管理器
import contextlib

@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write

    def reverse_write(text):
        original_write(text[::-1])

    sys.stdout.write = reverse_write
    yield 'JABBERWOCKY' # 前后分隔
    sys.stdout.write = original_write


contextlib.contextmanager 装饰器会把函数包装成实现了 \_\_enter__/\_\_exit__ 协议的类

这个类的  \_\_enter__ 方法有如下作用:
1. 调用生成器函数，保存生成器对象(这里把它称为 gen)
2. 调用 next(gen)，执行到 yield 关键字所在的位置
3. 返回 next(gen) 产出的值，绑定到 as 语句的目标变量上

with 块中止时，\_\_exit__ 方法会做以下几件事:
1. 检查有没有把异常传给 exc_type；如果有，调用 gen.throw(exception)，在生成器函数定义体包含 yield 关键字的那一会抛出异常
2. 否则，调用 next(gen)，继续执行 yield 之后的代码

上面的代码并没有去处理异常，如果中途抛出异常挑出则会导致 sys.stdout.write 函数无法恢复  
下面的代码会修复这个问题

In [ ]:
# 使用 @contextmanager 实现异常处理的上下文管理器
import contextlib

@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write

    def reverse_write(text):
        original_write(text[::-1])

    sys.stdout.write = reverse_write

    msg = ''
    try:
        yield 'JABBERWOCKY' # 前后分隔
    except ZeroDivisionError:
        msg = 'Please DO NOT divide by zero'
    finally:
        sys.stdout.write = original_write
        if msg:
            print(msg)